In [1]:
import tensorflow as tf
from tensorflow import keras

import tensorflow_datasets as tfds

In [2]:
(train_data, test_data), info = tfds.load(
    'imdb_reviews/subwords8k',
    split=(tfds.Split.TRAIN, tfds.Split.TEST),
    with_info=True, as_supervised=True
)





Shuffling and writing examples to /Users/land/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVE7FFZ/imdb_reviews-train.tfrecord
Shuffling and writing examples to /Users/land/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVE7FFZ/imdb_reviews-test.tfrecord
Shuffling and writing examples to /Users/land/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0.incompleteVE7FFZ/imdb_reviews-unsupervised.tfrecord
Dataset imdb_reviews downloaded and prepared to /Users/land/tensorflow_datasets/imdb_reviews/subwords8k/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data

<DatasetV1Adapter shapes: ((None,), ()), types: (tf.int64, tf.int64)>

In [5]:
train_batches = train_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))
test_batches = test_data.shuffle(1000).padded_batch(10, padded_shapes=([None],[]))

In [6]:
train_batch, train_labels = next(iter(train_batches))
train_batch.numpy()

array([[  12, 1694, 4406, ...,    0,    0,    0],
       [  19,   32,  265, ...,    0,    0,    0],
       [  62,    9,    1, ...,    0,    0,    0],
       ...,
       [  19, 7301, 6304, ...,  108,  336, 7975],
       [ 274,  732,   61, ...,    0,    0,    0],
       [  19, 2031,   18, ...,    0,    0,    0]])

In [7]:
encoder = info.features['text'].encoder
encoder.subwords[:20]

['the_',
 ', ',
 '. ',
 'a_',
 'and_',
 'of_',
 'to_',
 's_',
 'is_',
 'br',
 'in_',
 'I_',
 'that_',
 'this_',
 'it_',
 ' /><',
 ' />',
 'was_',
 'The_',
 'as_']

In [8]:
embedding_dim=16

model = keras.Sequential([
  keras.layers.Embedding(encoder.vocab_size, embedding_dim),
  keras.layers.GlobalAveragePooling1D(),
  keras.layers.Dense(16, activation='relu'),
  keras.layers.Dense(1)
])

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 16)          130960    
_________________________________________________________________
global_average_pooling1d (Gl (None, 16)                0         
_________________________________________________________________
dense (Dense)                (None, 16)                272       
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 17        
Total params: 131,249
Trainable params: 131,249
Non-trainable params: 0
_________________________________________________________________


In [9]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [10]:
history = model.fit(
    train_batches,
    epochs=10,
    validation_data=test_batches, validation_steps=20)

Epoch 1/10
2500/2500 [==============================] - 9s 4ms/step - loss: 0.5086 - accuracy: 0.6961 - val_loss: 0.0000e+00 - val_accuracy: 0.0000e+00